![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### 5.1 - ML loyiha qadamlari

## Ma`lumotlarni ML uchun tayyolrash

<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

## 3-QADAM. Ma'lumotlarni Machine Learning uchun tayyorlash
Keyingi katta qadam ma'lumotlarni ML uchun tayyorlash. Avvalgi qadamlarda biz ma'lumotlarga to'g'ridan-to'g'ri ishlov berib, ularni tahlil qildik. O'zimiz uchun ba'zi kerakli xulosalarni oldik. Qaysidur ustunlardagi qiymatlarni o'zgartirtish, yangi ustunlar yaratish kabi ishlarni rejalashtirib oldik. Umuman olganda, ma'lumotlarg bo'lgan talabni shakllantridik. Navbat shu talablar asosida ma'lumotlarni tayyorlash.

Buning uchun endi biz ma'lumotlar bilan to'g'ridan-to'g'ri emas, funksiyalar orqali ishlaymiz. Ya'ni, avvalgi qismda bajargan barcha jarayonlarnbi funksiyalar yordamida avtomatlashtiramiz.
Bunday qilishimizdan maqsad esa, kelajakda yangi ma'lumotlarga ishlov berishda tayyor funksiyani ishga tushirish orqali ma'lumotlarni bittada bizga kerak ko'rinishga keltirib olamiz.

In [1]:
import pandas as pd
import numpy as np
import sklearn # scikit-learn kutubxonasi

Boshlanishiga, datasetni qayta yuklaymiz.

In [2]:
# Onlayn dataset joylashgan manzilini ko'rsatamiaz
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Ma'lumotlarni train va test setga ajratamiz.
1. Oddiy (muvozanatsiz) train/test set

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

2. Muvozanatli (Stratified) train/test set

In [4]:
df['income_cat'] = pd.cut(df['median_income'], bins=[0., 1.5, 3.0, 4.5, 6.0, np.inf], labels=[1,2,3,4,5])

from sklearn.model_selection import StratifiedShuffleSplit
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# stratified_split.split funksiyasi indekslar qaytaradi
for train_index, test_index in stratified_split.split(df,df['income_cat']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

strat_train_set.drop('income_cat', axis=1, inplace=True)
strat_test_set.drop('income_cat', axis=1, inplace=True)

3. Ishlash uchun muvozanatli setlarni tanlaymiz.

Bu safar `label` (basorat qilishimiz kerak bo'lgan) ustun `median_house_value` ni ham alohida ajratamiz.
Bunga sabab, qolgan ustunlarga qo'llangan o'zgarishlarni labelga ham qo'llamaslik.

In [5]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

![Imgur](https://i.imgur.com/lO0yL15.png)

### 3.1 Tozalash

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   longitude           20640 non-null  float64 
 1   latitude            20640 non-null  float64 
 2   housing_median_age  20640 non-null  float64 
 3   total_rooms         20640 non-null  float64 
 4   total_bedrooms      20433 non-null  float64 
 5   population          20640 non-null  float64 
 6   households          20640 non-null  float64 
 7   median_income       20640 non-null  float64 
 8   median_house_value  20640 non-null  float64 
 9   ocean_proximity     20640 non-null  object  
 10  income_cat          20640 non-null  category
dtypes: category(1), float64(9), object(1)
memory usage: 1.6+ MB


Esingizda bo'lsa, `df.info()` yordamida `total_bedrooms ` ustunida ba'zi qatorlarda qiymatlar yo'q (`NaN`) ekanini aniqlagan edik.

Ushbu qatorlar bizga halaqit bermasligi uchun oldimizda 3 ta yo'l bor:
1. Qatorlarni (hududlarni) o'chirib tashlash
2. Bu ustunni umuman tashlab yuborish
3. `NaN` qiymatlarni to'ldirish (`0`, o'rtacha yoki median qiymat bilan)

Bulardan qay birini tanlash, loyihadan loyihaga qarab o'zgaradi. Agar birinchi ikki usulni tansalangiz, buning yo'li oson:

In [7]:
# 1-variant. Qatorlarni tashlab yuborish
housing.dropna(subset=['total_bedrooms']) # inplace=True qilishni unutmang

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,<1H OCEAN
12661,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,INLAND
19263,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,<1H OCEAN
19140,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,<1H OCEAN


In [8]:
# 2-variant. Ustunni tashlab yuborish
housing.drop("total_bedrooms", axis=1) # inplace=True qilishni unutmang

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,1837.0,580.0,4.4964,<1H OCEAN
...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,2026.0,1001.0,5.0900,<1H OCEAN
12661,-121.42,38.51,15.0,7901.0,4769.0,1418.0,2.8139,INLAND
19263,-122.72,38.44,48.0,707.0,458.0,172.0,3.1797,<1H OCEAN
19140,-122.70,38.31,14.0,3155.0,1208.0,501.0,4.1964,<1H OCEAN


In [9]:
# 3-variant. Median qiymat bilan to'ldirish
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median) # inplace=True qilishni unutmang

,total_bedrooms
12655,797.0
15502,855.0
2908,310.0
14053,519.0
20496,646.0
...,...
15174,1231.0
12661,1422.0
19263,166.0
19140,580.0


Biz 3-variantni tanlaymiz va buni biroz takomillashtiramiz.

Gap shundaki, bu safar bizga kelgan datasetda `total_bedrooms` ustunida `NaN` qiymatlar bor edi, lekin kelgusida yangi ma'lumotlar kelganda, boshqa, deylik `population` ustunida `NaN` qiymatlar bo'lishi mumkin. Biz yuqorida ma'lumotlarga ishlov berishni avtomatlashtirish haqida gapirdik, demak bizning funksiyamiz, har qanday ustundagi `NaN` qiymatlarni to'ldirib ketishi kerak.

Buning uchun `sklearn` tarkibida maxsus `SimpleImputer` funksiyasi mavjud.

In [10]:
# SimpleImputer klassini chaqiramiz
from sklearn.impute import SimpleImputer
# SimpleImputer klassidan yangi obyekt yaratamiz va NaN qiymatlarni median qiymat bilan to'ldirishni ko'rsatamiz
imputer = SimpleImputer(strategy="median")

Dataset uchun median qiymatni hisoblash uchun `imputer` obyekti ichidagi `.fit()` metodiga murojaat qilamiz.
Dataset ustunlariga e'tibor bersak, oxirgi `ocean_proximity` ustuni sonli emas. `imputer` esa faqatgina sonli qiymatlar bilan ishlaydi (sonsiz qiymatlarnini o'rta qiymati mavjud emas).

In [11]:
imputer.fit(housing)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'INLAND'

Yuqoridagi xatoning oldini olish uchun, sonli ustunlarni alohida ajratib olamiz:

In [12]:
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

SimpleImputer(strategy='median')

Hisoblangan median qiymatlar `statistics_` atrributida saqlanadi:

In [13]:
imputer.statistics_

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

Yuqorida har bir ustun uchun median qiymatni ko'rishingiz mumkin

Datasetdagi `NaN` qiymatlarni hisoblangan median qiymatlar bilan to'ldirish uchun `.transform()` funksiyasiga murojaat qilamiz:

In [14]:
X = imputer.transform(housing_num)

Natijani `X` o'zgaruvchiga saqladik.

In [15]:
type(X)

numpy.ndarray

 `.transform()` funksiyasi numpy array qaytaradi. Arrayni qaytarib dataframega o'tkazamiz:

In [16]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [17]:
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964


### 3.2 Matnli ustunlarni shakl almashtirish
Datasetimizning `ocean_proximity` ustuni matnli ustun. Keling boshlanishga bu ustunni ajratib olamiz.

In [18]:
housing_cat = housing[['ocean_proximity']]

In [19]:
housing_cat["ocean_proximity"].head(10)

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN
1481,NEAR BAY
18125,<1H OCEAN
5830,<1H OCEAN
17989,<1H OCEAN
4861,<1H OCEAN


 ML algoritmlar sonlar bilan ishlaydi. Shuning uchun bu ustunni ham sonlarga o'zgartirishimiz kerak.

Buning 2 usuli bor:
1. Har bir matnni son bilan almashtirish.

Buning uchun sklearn tarkibida `OrdinalEncoder`dan foydalanamiz:

In [20]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

Ko'rib turganingizdek, matnlar sonlar bilan almashdi. Bu usul juda qulay lekin o'ziga yarasha kamchilig bor. Gap shundaki ML algoritm ikki yaqin songa mos keluvchi qatorlar bir-biriga yaqin deb qabul qiladi. Agar bu haqiqatdan shunday bo'lsa, model to'g'ri ishlaydi, lekin bu narsa haqiqatga to'g'ri kelmasa kelajakda model ishlashida noaniqlik baland bo'ladi.

In [21]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

Bizda, ahamiyat bersangiz qo'shni qiymatlar (matnlar) bir-biriga yaqin emas. `<1H OCEAN` va `INLAND` ga `0.0` va `1.0` qiymatlari berilgan, vaholanki, mantiqan o'ylansa bu ikkisi bir-biridan uzoq. `NEAR BAY`, `NEAR OCEAN` qiymatlari esa aksincha, `<1H OCEAN`dan uzoqlashib ketgan.

2. `OneHotEncoder` - usuli yordamida har bir noyob qiymat alohida ustunga aylanadi va mos keluvchi ustunga 1, qolganlariga 0 qo'yiladi. Tushunarli bo'lishi uchun keyingi kodga ahamiyat bering.

In [22]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

Yuqorida nima bo'ldi, bizda 5 ta kategoriya bor edi ('<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN').

OneHotEncoder 5 ta ustundan iborat yangi matrisa yaratdi. Har bir ustun yuqoridagi bitta kategoriyaga mos keladi: 1-ustun:'<1H OCEAN', 2-ustun:'INLAND' va hokazo.
Endi, har bir qatordagi matnning qiymatiga qarab mos keluvchi ustunga 1 qo'yiladi (qolgani 0 bo'ladi).

Masalan, `housing_cat` dataset 1-qatorini ko'rsak, `<1H OCEAN` qiymati turibdi, demak matrisaning 1-ustuni `1` qolganlari `0` bo'ladi. Buni yuoqridagi katakda ko'rishimiz mumkin.

In [23]:
housing_cat.head(10)

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN
1481,NEAR BAY
18125,<1H OCEAN
5830,<1H OCEAN
17989,<1H OCEAN
4861,<1H OCEAN


Birinchi yoki ikkinchi usuldan qayerda foydalanish sizga havola. Agar matnli qiymatlarning turi judayam ko'p bo'lsa 2-usul datasetning kattalshib ketishiga sabab bo'ladi. Yuqorida, bizda 5 ta kategoriya bo'lgani uchun, datasetga yana 5 ta ustun qo'shishimiz kerak, bizda 16512 qator borligini hisobga olsak bu 82560 ta qo'shimcha qiymat degani.

Demak kategoriyalar soni oshgan sari, ishlov berish kerak bo'lgan ustunlar ham oshib ketadi bu esa ortiqcha resurs ishlatilishiga va qo'shimcha moliyaviy harajatlarga olib keladi.

### 3.3 Transfomer

Yuqorida ma'lumotlarg ishlov berishda biz `sklearn` tarkibidagi tayyor obyektlar va ulardagi `.fit()` yoki `.fit_transform()` funksiyalariga murojaat qildik.
Bu funksiyalar (`imputer`, `OneHotEncoder` va hokazo) **transformer**lar deb ataladi.

Umuman olganda `sklearn` tarkibidagi obyektlar uch turli bo'ladi:
1. **Estimator** - tahmin qiluvchi (baholovchi) funksiyalar. Bunday obyektlar `.fit()` metodiga ega bo'ladi, bu metod berilgan ma'lumotlardan qandaydur qiymatni hisoblaydi (masalan, avvalroq biz `imputer.fit()` yordamida median qiymatni hisobladik)
2. **Transformer** - ham tahmin qilib ham ma'lumotga ishlov beruvchi obyekt. Bunday obyektlar `.fit_transform()` metodiga ega bo'ladi, ya'ni avval qandaydur qiymatnbi hisoblaydi, keyin esa hisoblashni ma'lumotlarga qo'llaydi. Yuoqirdagi `OrdinalEncoder` va `OneHotEncoder` shunday obyektlardan.
3. **Predictor** - bashorat qiluvchi obyektlar. Bunday obyektlar `.predict()` metodiga ega bo'ladi. Ular biz bergan ma'lumotlar asosida bashoratlar qaytaradi. Dars davomida predictor obyektlar bilan ko'p ishlaymiz.

Obyektlarning afzalligi, ular hisob kitob natijalarini ichki attributrlarda saqlab qoladi. Masalan, yuqorida ko'rganimiz `imputer.statistics_`.

Keling endi o'zimiz ham transformer yozishni o'rganamiz.

Esingizda bo'lsa, avvalgi darsimizda biz ma'lumotlarg yangi 2 ustun qo'shdik: `rooms_per_household` va `bedrooms_per_room`.

Keling berilgan ma'lumotlarga shu ikki ustunni avtomat ravishda qo'shadigan transformer yasaymiz.

Buning uchun `sklearn` tarkibida `BaseEstimator` va `TransformerMixin` klasslaridan meros olib yangi klass yasaymiz va bu klassimizga kabi `fit()` va `transform()` metodlarini qo'shamiz:

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin

# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [25]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=True)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs[0,:]

array([-121.46, 38.52, 29.0, 3873.0, 797.0, 2237.0, 706.0, 2.1736,
       'INLAND', 5.485835694050992, 3.168555240793201,
       0.20578363026077975], dtype=object)

In [26]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs[0,:]

array([-121.46, 38.52, 29.0, 3873.0, 797.0, 2237.0, 706.0, 2.1736,
       'INLAND', 5.485835694050992, 3.168555240793201], dtype=object)

### 3.4 Standartizasiya va normalizasiya

Ahamiyat bersangiz, bizdagi ustunlardagi qiymatlar diapazoni turlicha, masalan:
- `housing_median_age` - `0-50` oralig'ida
- `total_rooms` esa `6-39000` oralig'ida
- `median_income` - `0.5-15` oralig'ida

Bularning hammasi kelgusi hisob kitoblarda algoritmlarning to'gri ishlashiga ta'sir ko'rsatadi.

Buning oldini olish uchun barcha ustundagi qiymatlarni bir diapazonga keltirib olish tavsiya qilinadi.

Buning uchun ML da min-max normallashtirish (_min-max scaling_) yoki standartlash (_standartization_) qo'llaniladi.

#### 3.4.1 Min-max scaling

Min-max scaling berilgan qiymatlarni 0-1 oralig'iga olib keladi.

Buning uchun biror ustundagi barcha qiymatlarning min ($x_{min}$) va max ($x_{max}$) qiymatlari topiladi. Keyin esa ustundagi har bir qiymat ($x$) quyidagi formula asosida hisoblab chiqiladi:

![](https://arshpreetsingh.files.wordpress.com/2017/03/normal.png)

Yuqoridagi vazifani bajarish uchun `sklearn` tarkibida maxsus `MinMaxScaler` obyekti bor. Keling, min-max scalingni bizdagi ma'lumotlarga qo'llab ko'ramiz:

In [27]:
from sklearn.preprocessing import MinMaxScaler
min_mix_scaler = MinMaxScaler()

min_mix_scaler.fit_transform(housing_num)

array([[0.28784861, 0.63549416, 0.54901961, ..., 0.06261386, 0.13144137,
        0.11542599],
       [0.70916335, 0.05844846, 0.11764706, ..., 0.05639172, 0.14301718,
        0.40257376],
       [0.52888446, 0.30074389, 0.84313725, ..., 0.01861039, 0.05563854,
        0.16379774],
       ...,
       [0.1623506 , 0.62699256, 0.92156863, ..., 0.0127526 , 0.0317401 ,
        0.18481124],
       [0.16434263, 0.61317747, 0.25490196, ..., 0.03377337, 0.09316654,
        0.25492752],
       [0.22011952, 0.78958555, 0.50980392, ..., 0.01743322, 0.03640777,
        0.18151474]])

Ko'rib turganingizdek, sonlar 0-1 oralig'iga keldi.

#### 3.4.2 Standard Scaler - standartlash
StandardScaler biroz boshqa usulda ishlaydi. Sonlarni standartlas hcuhn bizga berilgan ustun bo'yicha o'rta qiymat ($\mu$) va standart cheklanish ($\sigma$) talab qilinadi. Har bir qatordag qiymat esa quydagicha hisoblanadi

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxeMnm1BSa8qyNkCB6wqiYa790XtNxCdM48k_be-9zESUvivv_nuY6SXk8fpJK_psGiA&usqp=CAU)

In [28]:
from sklearn.preprocessing import StandardScaler
standart_scaler = StandardScaler()
standart_scaler.fit_transform(housing_num)

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.73260236,
         0.55628602, -0.8936472 ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.53361152,
         0.72131799,  1.292168  ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ..., -0.67467519,
        -0.52440722, -0.52543365],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ..., -0.86201341,
        -0.86511838, -0.36547546],
       [-1.56080303,  1.2492109 , -1.1653327 , ..., -0.18974707,
         0.01061579,  0.16826095],
       [-1.28105026,  2.02567448, -0.13148926, ..., -0.71232211,
        -0.79857323, -0.390569  ]])

In [29]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN


In [30]:
housing_onehot = pd.get_dummies(housing['ocean_proximity'])
housing_onehot

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
12655,False,True,False,False,False
15502,False,False,False,False,True
2908,False,True,False,False,False
14053,False,False,False,False,True
20496,True,False,False,False,False
...,...,...,...,...,...
15174,True,False,False,False,False
12661,False,True,False,False,False
19263,True,False,False,False,False
19140,True,False,False,False,False


In [31]:
housing_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964
